# IMDB Movie reviews sentiment classification in Keras

In [2]:
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt

In [3]:
# Loading the data
(x_train, y_train), (x_test, y_test) = imdb.load_data(
                                        num_words=1500, 
                                        index_from=3,
                                        skip_top=10)

print(x_train.shape,y_train.shape)
print(x_test.shape,y_train.shape)

(25000,) (25000,)
(25000,) (25000,)


In [4]:
# Encoded text sample as sequence of indices
print(x_train[0])
# Corresponding label
print(y_train[0])

# Original Text
word_to_idx = keras.datasets.imdb.get_word_index()
word_to_idx = {k:v+3 for k,v in word_to_idx.items()}
word_to_idx["<PAD>"] = 0
word_to_idx["<START>"] = 1
word_to_idx["<SKIP>"] = 2
idx_to_word = {value:key for key,value in word_to_idx.items()}

print(' '.join(idx_to_word[idx] for idx in x_train[0]))

[2, 14, 22, 16, 43, 530, 973, 2, 1385, 65, 458, 2, 66, 2, 2, 173, 36, 256, 2, 25, 100, 43, 838, 112, 50, 670, 2, 2, 35, 480, 284, 2, 150, 2, 172, 112, 167, 2, 336, 385, 39, 2, 172, 2, 1111, 17, 546, 38, 13, 447, 2, 192, 50, 16, 2, 147, 2, 19, 14, 22, 2, 2, 2, 469, 2, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 2, 22, 17, 515, 17, 12, 16, 626, 18, 2, 2, 62, 386, 12, 2, 316, 2, 106, 2, 2, 2, 2, 16, 480, 66, 2, 33, 2, 130, 12, 16, 38, 619, 2, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 2, 22, 12, 215, 28, 77, 52, 2, 14, 407, 16, 82, 2, 2, 2, 107, 117, 2, 15, 256, 2, 2, 2, 2, 2, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 2, 2, 2, 1029, 13, 104, 88, 2, 381, 15, 297, 98, 32, 2, 56, 26, 141, 2, 194, 2, 18, 2, 226, 22, 21, 134, 476, 26, 480, 2, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 2, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 2, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1
<SKIP> this film was just brilliant casting <SKIP> scenery story direction <SKIP> really <SKIP> <SKIP> part t

In [5]:
# One-hot encoding the input
tokenizer = Tokenizer(num_words=1500)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train[0])

[0. 0. 1. ... 0. 0. 0.]


In [6]:
# One-hot encode the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


In [7]:
model = Sequential()
model.add(Dense(1000, activation='relu', input_dim=1500))
model.add(Dropout(0.7))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compiling model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              1501000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               200200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                10050     
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 102       
Total para

In [8]:
# Train model
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_test, y_test), 
          verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
 - 10s - loss: 0.4970 - acc: 0.7719 - val_loss: 0.3408 - val_acc: 0.8696
Epoch 2/5
 - 7s - loss: 0.3835 - acc: 0.8592 - val_loss: 0.3315 - val_acc: 0.8702
Epoch 3/5
 - 7s - loss: 0.3594 - acc: 0.8714 - val_loss: 0.3801 - val_acc: 0.8685
Epoch 4/5
 - 7s - loss: 0.3425 - acc: 0.8789 - val_loss: 0.3381 - val_acc: 0.8711
Epoch 5/5
 - 7s - loss: 0.3405 - acc: 0.8787 - val_loss: 0.3503 - val_acc: 0.8732


In [9]:
# Test set accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.87324
